# Somers' D

Author: https://github.com/deburky

## Some basic examples

In [1]:
from scipy.stats import somersd
import numpy as np

# Create the data
data = [[3, 1],
        [5, 7],
        [2, 6]]

# Convert the data to a NumPy array
table = np.array(data).T
print(table)

res = somersd(table)
print(res.statistic)
# 0.2727272727272727

[[3 5 2]
 [1 7 6]]
0.34285714285714286


In [2]:
import numpy as np
from scipy.stats import somersd

# Create the data
data = [[140, 2],
        [180, 35],
        [93, 87]]

# Transpose the table
table = np.array(data).T
print(table)

# For Dxy, y is the row, x is the column
res = somersd(table)
print(f"{res.statistic:.4f}")
# [[140 180  93]
# [  2  35  87]]
# 0.5651

[[140 180  93]
 [  2  35  87]]
0.5651


In [3]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Define the distribution
distribution = {
    'Rating 1': [140, 2],
    'Rating 2': [180, 35],
    'Rating 3': [93, 87]
}

# Create the dataset
data = []

for rating, counts in distribution.items():
    rating_value = int(rating.split()[1])
    zeros = [0] * counts[0]
    ones = [1] * counts[1]
    data.extend([(rating_value, value) for value in zeros + ones])

# Convert to DataFrame
df = pd.DataFrame(data, columns=['Rating', 'Default'])
somers_d = roc_auc_score(df['Default'], df['Rating']) * 2 - 1
print(f"{somers_d:.4f}")
# 0.5651

0.5651


## Implementation of Somers' D from scratch

In [4]:
import numpy as np
from typing import Tuple
import itertools

def somers_d_from_matrix(matrix: np.ndarray) -> Tuple[float, int, int, int]:
    """
    Calculate Somers' D from a given matrix.
    
    Parameters:
    matrix (np.ndarray): a matrix where rows and columns represent different groups.

    Returns:
    Tuple[float, int, int, int]: Somers' D, number of concordant pairs, number of discordant pairs, and total pairs.
    
    Comments:
    When we transpose the matrix, calculation becomes faster when we sum cols, not rows.
    """
    
    # ensure the matrix is a numpy array
    matrix = np.array(matrix).astype(int)
    
    # Calculate concordant and discordant pairs
    concordant_pairs = 0
    discordant_pairs = 0
    
    for i, j in itertools.combinations(range(len(matrix)), 2):
        concordant_pairs += np.sum(np.triu(np.outer(matrix[i], matrix[j]), 1))
        discordant_pairs += np.sum(np.triu(np.outer(matrix[j], matrix[i]), 1))
    
    # Calculate total pairs
    col_sums = np.sum(matrix, axis=0)
    total_pairs = np.sum(np.triu(np.outer(col_sums, col_sums), 1))

    # # # Calculate total pairs using row sums
    # row_sums = np.sum(matrix, axis=1)
    # total_pairs = np.sum(np.triu(np.outer(row_sums, row_sums), 1))
    
    somers_d = (concordant_pairs - discordant_pairs) / total_pairs

    return somers_d, concordant_pairs, discordant_pairs, total_pairs

# Define the matrix as per your example
# matrix = np.array([
#     [140, 180, 93],
#     [2, 35, 87],
#     [10, 12, 10]
# ])

matrix = np.array([
    [140, 2],
    [180, 35],
    [93, 87],
    [65, 55]
]).T

print("Input matrix:")
print(matrix)
print('---' * 8)

# Calculate Somers' D
somers_d, concordant, discordant, total_pairs = somers_d_from_matrix(matrix)

print(f"Concordant Pairs: {concordant}")
print(f"Discordant Pairs: {discordant}")
print(f"Total Pairs: {total_pairs}")
print('---' * 8)

%time somers_d = somers_d_from_matrix(matrix.T)[0]
print(f"Somers D with numpy: {somers_d:.4f}")

# Calculate the Somers' D statistic with scipy
from scipy.stats import somersd
%time somers_d_sc = somersd(matrix).statistic
print(f"Somers D with scipy: {somers_d_sc:.4f}")

Input matrix:
[[140 180  93  65]
 [  2  35  87  55]]
------------------------
Concordant Pairs: 55455
Discordant Pairs: 11861
Total Pairs: 159230
------------------------
CPU times: user 204 µs, sys: 18 µs, total: 222 µs
Wall time: 212 µs
Somers D with numpy: 0.5095
CPU times: user 285 µs, sys: 55 µs, total: 340 µs
Wall time: 319 µs
Somers D with scipy: 0.5095


## Binary problem

In [5]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.stats import somersd

X, y = make_classification(
    n_samples=100, n_features=20, n_informative=10, n_classes=2, random_state=42
)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(random_state=42).fit(X_train, y_train)
y_pred = model.predict_proba(X_test)[:, 1]

from scipy.stats import somersd

table = pd.crosstab(y_test, y_pred).values
somers_d = somersd(table).statistic # scipy
somers_dxy = somers_d_from_matrix(table.T)[0] # numpy
auc = roc_auc_score(y_test, y_pred) # sklearn
somers_d_auc = 2 * auc - 1

print(f"Somers' D (scipy): {somers_d:.4f}")
print(f"Somers' D (numpy): {somers_dxy:.4f}")
print(f"Somers' D (roc_auc): {somers_d_auc:.4f}")

Somers' D (scipy): 0.6566
Somers' D (numpy): 0.6566
Somers' D (roc_auc): 0.6566


## Non-binary problem

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb

np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

lgd_dataset = pd.read_csv('lgd.csv')
features = ['LTV', 'purpose1', 'event']
target = 'lgd_time'

# lgd_dataset = pd.read_csv('lgd_dataset.csv')
# target = lgd_dataset.columns[-1]
# features = lgd_dataset.columns[:-1]

X = lgd_dataset[features]
y = lgd_dataset[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = xgb.XGBRFRegressor(random_state=0)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

table = pd.crosstab(y_test, predictions).values

%time somers_dxy = somers_d_from_matrix(table.T)[0]
print(f"Somers' D (numpy): {somers_dxy:.4f}")

# time_start = time.time()
%time somers_d = somersd(table).statistic
# time_scipy = time.time() - time_start
print(f"Somers' D (scipy): {somers_d:.4f}")
# print(f"Time taken by SciPy: {time_scipy:.4f} seconds")

CPU times: user 2.91 s, sys: 3.49 ms, total: 2.91 s
Wall time: 2.92 s
Somers' D (numpy): 0.6094
CPU times: user 3.9 s, sys: 1.26 ms, total: 3.9 s
Wall time: 3.91 s
Somers' D (scipy): 0.6094


In [7]:
from time import time

table = pd.crosstab(y_test, predictions).values # 340 x 160

time_start = time()
somers_d = somersd(table).statistic
time_scipy = time() - time_start
print(f"Time scipy: {time_scipy:.4f} seconds")
print(f"Somers' D (scipy): {somers_d:.4f}")

time_start = time()
somers_dxy = somers_d_from_matrix(table.T)[0]
time_numpy = time() - time_start
print(f"Somers' D (numpy): {somers_dxy:.4f}")
print(f"Time numpy: {time_numpy:.4f} seconds")

# calculate speed improvement
difference = (time_scipy - time_numpy) / time_numpy
print(f"Time improvement: {difference:.0%}")

Time scipy: 3.9258 seconds
Somers' D (scipy): 0.6094
Somers' D (numpy): 0.6094
Time numpy: 3.1535 seconds
Time improvement: 24%


## Different models

In [18]:
%%time
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import xgboost as xgb
import catboost as ctb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from rich.table import Table


def somers_d_score(X, y):
    """
    Function to calculate Somers' D from a matrix.
    We need to have y as the row and x as the column.
    To speed up, we use a transpose of the matrix.
    """
    matrix = pd.crosstab(X, y).values
    return somers_d_from_matrix(matrix.T)[0]

clfs = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(random_state=0),
    'MLP': MLPRegressor(random_state=0),
    'Random Forest': RandomForestRegressor(random_state=0),
    'XGB Random Forest': xgb.XGBRFRegressor(random_state=0),
    'HistGradientBoosting': HistGradientBoostingRegressor(random_state=0),
    'XGBoost': xgb.XGBRegressor(random_state=0),
    'LightGBM': lgb.LGBMRegressor(verbose=0, random_state=0),
    'CatBoost': ctb.CatBoostRegressor(random_state=0, verbose=0),
}

results = {'model': [], 'somers_d': []}

for name, model in clfs.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    preds_and_target = pd.DataFrame({'LGD': y_test, 'pred': predictions})
    somers_d_model = somers_d_score(preds_and_target['LGD'], preds_and_target['pred'])
    results['model'].append(name)
    results['somers_d'].append(somers_d_model)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='somers_d', ascending=False)

# show Rich table with results
table = Table(title="Model evaluation")
table.add_column("Model", justify="right", style="royal_blue1", no_wrap=True)
table.add_column("Dxy", style="royal_blue1")

for model, somers_d in zip(results_df['model'], results_df['somers_d']):
    table.add_row(model, f"{somers_d:.4f}")
    
display(table)

        Model evaluation         
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃                Model ┃ Dxy    ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│                  MLP │ 0.6133 │
│ HistGradientBoosting │ 0.6112 │
│    XGB Random Forest │ 0.6094 │
│             CatBoost │ 0.6055 │
│                Ridge │ 0.5978 │
│               Linear │ 0.5977 │
│             LightGBM │ 0.5955 │
│              XGBoost │ 0.5942 │
│        Random Forest │ 0.5796 │
└──────────────────────┴────────┘

CPU times: user 2min 30s, sys: 14.6 s, total: 2min 45s
Wall time: 2min 33s


In [19]:
%%time
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import xgboost as xgb
import catboost as ctb
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from rich.table import Table

from scipy.stats import somersd

clfs = {
    'Linear': LinearRegression(),
    'Ridge': Ridge(random_state=0),
    'MLP': MLPRegressor(random_state=0),
    'Random Forest': RandomForestRegressor(random_state=0),
    'XGB Random Forest': xgb.XGBRFRegressor(random_state=0),
    'Gradient Boosting': HistGradientBoostingRegressor(random_state=0),
    'XGBoost': xgb.XGBRegressor(random_state=0),
    'LightGBM': lgb.LGBMRegressor(verbose=0, random_state=0),
    'CatBoost': ctb.CatBoostRegressor(random_state=0, verbose=0),
}

results = {'model': [], 'somers_d': []}

for name, model in clfs.items():
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    preds_and_target = pd.DataFrame({'LGD': y_test, 'pred': predictions})
    matrix = pd.crosstab(preds_and_target['LGD'], preds_and_target['pred']).values
    somers_d_model = somersd(matrix).statistic
    results['model'].append(name)
    results['somers_d'].append(somers_d_model)

results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='somers_d', ascending=False)

# show Rich table with results
table = Table(title="Model evaluation")
table.add_column("Model", justify="right", style="royal_blue1", no_wrap=True)
table.add_column("Dxy", style="royal_blue1")

for model, somers_d in zip(results_df['model'], results_df['somers_d']):
    table.add_row(model, f"{somers_d:.4f}")
    
display(table)

       Model evaluation       
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃             Model ┃ Dxy    ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│               MLP │ 0.6133 │
│ Gradient Boosting │ 0.6112 │
│ XGB Random Forest │ 0.6094 │
│          CatBoost │ 0.6055 │
│             Ridge │ 0.5978 │
│            Linear │ 0.5977 │
│          LightGBM │ 0.5955 │
│           XGBoost │ 0.5942 │
│     Random Forest │ 0.5796 │
└───────────────────┴────────┘

CPU times: user 3min 7s, sys: 7.76 s, total: 3min 15s
Wall time: 3min 3s


## OptBinning example

OptBinning allows to work with continuous outcomes, here we measure Somers' D for only one variable.

In [65]:
from optbinning import ContinuousOptimalBinning

variable_name = 'LTV'
x = X_train[variable_name].values
y = y_train.values

optb = ContinuousOptimalBinning(
    name=variable_name,
    dtype="numerical",
    min_bin_size=1e-1,
    max_n_bins=5
)

optb.fit(x, y)

# build a binning table
binning_table = optb.binning_table.build()
predicted_scores = optb.transform(X_test[variable_name])
predicted_scores = pd.Series(predicted_scores, name='lgd_pred', index=y_test.index)
table = pd.crosstab(y_test, predicted_scores).values

somers_d = somersd(table).statistic
print(f"Somers' D (scipy): {somers_d:.4f}")

somers_dxy = somers_d_from_matrix(table.T)[0]
print(f"Somers' D (numpy): {somers_dxy:.4f}")

Somers' D (scipy): 0.2625
Somers' D (numpy): 0.2625


In [66]:
pd.crosstab(y_test, predicted_scores)

lgd_pred,0.091234,0.130317,0.190938,0.420531,0.545768
lgd_time,,,,,
0.000010,41,21,71,10,6
0.000018,1,0,0,0,0
0.000135,0,0,1,0,0
0.000717,0,0,1,0,0
0.001137,1,0,0,0,0
...,...,...,...,...,...
0.994413,0,0,0,0,1
0.994834,0,0,0,1,0
0.995180,0,0,1,0,0
